In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt
import time

# Using mediapipe holistic, full body pose estimation

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_custom_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(255, 255, 204), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(225, 225, 225), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(204, 153, 255), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    

# Get landmarks keypoints from frames

In [6]:
def get_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
  
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3) 

    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

# Load data

In [171]:
DATA_PATH = os.path.join('points_Data')

actions = np.array(['hello', 'thank you', 'i am', 'w'])
#['##', hello', 'thank you', 'i love you', 'i am', 'w', 'a', 'i', 'l']
no_sequences = 50

sequence_length = 30

In [172]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [173]:
label_map = {label:num for num, label in enumerate(actions)}

In [174]:
actions

array(['hello', 'thank you', 'i am', 'w'], dtype='<U9')

In [177]:
sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in  range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [178]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [179]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [180]:
len(sequences)

200

# Tensor Flow Model

In [181]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

In [182]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [183]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [184]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [185]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
6/6 [==============================] - 4s 82ms/step - loss: 1.4766 - categorical_accuracy: 0.3368
Epoch 2/2000
6/6 [==============================] - 0s 80ms/step - loss: 3.2239 - categorical_accuracy: 0.2053
Epoch 3/2000
6/6 [==============================] - 0s 83ms/step - loss: 1.4902 - categorical_accuracy: 0.2526
Epoch 4/2000
6/6 [==============================] - 0s 83ms/step - loss: 0.9607 - categorical_accuracy: 0.4842
Epoch 5/2000
6/6 [==============================] - 0s 81ms/step - loss: 0.6903 - categorical_accuracy: 0.6789
Epoch 6/2000
6/6 [==============================] - 0s 81ms/step - loss: 0.9586 - categorical_accuracy: 0.6579
Epoch 7/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.1783 - categorical_accuracy: 0.5158
Epoch 8/2000
6/6 [==============================] - 0s 81ms/step - loss: 0.9265 - categorical_accuracy: 0.4789
Epoch 9/2000
6/6 [==============================] - 0s 80ms/step - loss: 0.8587 - categorical_accuracy: 0.6053
E

6/6 [==============================] - 0s 82ms/step - loss: 0.8082 - categorical_accuracy: 0.5737
Epoch 75/2000
6/6 [==============================] - 0s 83ms/step - loss: 0.5529 - categorical_accuracy: 0.7737
Epoch 76/2000
6/6 [==============================] - 1s 85ms/step - loss: 0.4346 - categorical_accuracy: 0.8737
Epoch 77/2000
6/6 [==============================] - 1s 84ms/step - loss: 0.4086 - categorical_accuracy: 0.9000
Epoch 78/2000
6/6 [==============================] - 0s 82ms/step - loss: 0.2437 - categorical_accuracy: 0.9789
Epoch 79/2000
6/6 [==============================] - 1s 85ms/step - loss: 0.1633 - categorical_accuracy: 0.9789
Epoch 80/2000
6/6 [==============================] - 0s 85ms/step - loss: 0.2545 - categorical_accuracy: 0.8684
Epoch 81/2000
6/6 [==============================] - 1s 84ms/step - loss: 0.2427 - categorical_accuracy: 0.9158
Epoch 82/2000
6/6 [==============================] - 1s 85ms/step - loss: 0.1362 - categorical_accuracy: 0.9526
Epoch 

6/6 [==============================] - 1s 86ms/step - loss: 1.9781e-04 - categorical_accuracy: 1.0000
Epoch 146/2000
6/6 [==============================] - 0s 84ms/step - loss: 1.8435e-04 - categorical_accuracy: 1.0000
Epoch 147/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.8009e-04 - categorical_accuracy: 1.0000
Epoch 148/2000
6/6 [==============================] - 0s 81ms/step - loss: 1.6599e-04 - categorical_accuracy: 1.0000
Epoch 149/2000
6/6 [==============================] - 0s 80ms/step - loss: 1.6325e-04 - categorical_accuracy: 1.0000
Epoch 150/2000
6/6 [==============================] - 0s 80ms/step - loss: 1.5875e-04 - categorical_accuracy: 1.0000
Epoch 151/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.5805e-04 - categorical_accuracy: 1.0000
Epoch 152/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.4090e-04 - categorical_accuracy: 1.0000
Epoch 153/2000
6/6 [==============================] - 0s 81ms/step - loss: 1.42

6/6 [==============================] - 0s 78ms/step - loss: 1.9160e-05 - categorical_accuracy: 1.0000
Epoch 216/2000
6/6 [==============================] - 0s 80ms/step - loss: 1.8640e-05 - categorical_accuracy: 1.0000
Epoch 217/2000
6/6 [==============================] - 0s 80ms/step - loss: 1.8310e-05 - categorical_accuracy: 1.0000
Epoch 218/2000
6/6 [==============================] - 0s 78ms/step - loss: 1.7545e-05 - categorical_accuracy: 1.0000
Epoch 219/2000
6/6 [==============================] - 0s 79ms/step - loss: 1.7335e-05 - categorical_accuracy: 1.0000
Epoch 220/2000
6/6 [==============================] - 0s 80ms/step - loss: 1.6708e-05 - categorical_accuracy: 1.0000
Epoch 221/2000
6/6 [==============================] - 0s 78ms/step - loss: 1.6196e-05 - categorical_accuracy: 1.0000
Epoch 222/2000
6/6 [==============================] - 0s 79ms/step - loss: 1.6059e-05 - categorical_accuracy: 1.0000
Epoch 223/2000
6/6 [==============================] - 0s 78ms/step - loss: 1.53

6/6 [==============================] - 0s 82ms/step - loss: 5.9099e-06 - categorical_accuracy: 1.0000
Epoch 286/2000
6/6 [==============================] - 0s 80ms/step - loss: 5.8610e-06 - categorical_accuracy: 1.0000
Epoch 287/2000
6/6 [==============================] - 0s 82ms/step - loss: 5.7750e-06 - categorical_accuracy: 1.0000
Epoch 288/2000
6/6 [==============================] - 1s 85ms/step - loss: 5.6947e-06 - categorical_accuracy: 1.0000
Epoch 289/2000
6/6 [==============================] - 0s 84ms/step - loss: 5.6772e-06 - categorical_accuracy: 1.0000
Epoch 290/2000
6/6 [==============================] - 1s 84ms/step - loss: 5.5624e-06 - categorical_accuracy: 1.0000
Epoch 291/2000
6/6 [==============================] - 0s 85ms/step - loss: 5.5103e-06 - categorical_accuracy: 1.0000
Epoch 292/2000
6/6 [==============================] - 0s 83ms/step - loss: 5.4501e-06 - categorical_accuracy: 1.0000
Epoch 293/2000
6/6 [==============================] - 0s 83ms/step - loss: 5.39

...

6/6 [==============================] - 0s 82ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1676/2000
6/6 [==============================] - 0s 81ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1677/2000
6/6 [==============================] - 0s 81ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1678/2000
6/6 [==============================] - 0s 83ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1679/2000
6/6 [==============================] - 1s 86ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1680/2000
6/6 [==============================] - 0s 83ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1681/2000
6/6 [==============================] - 0s 84ms/step - loss: 5.6468e-09 - categorical_accuracy: 1.0000
Epoch 1682/2000
6/6 [==============================] - 0s 83ms/step - loss: 6.2742e-09 - categorical_accuracy: 1.0000
Epoch 1683/2000
6/6 [==============================] - 1s 83ms/step - lo

6/6 [==============================] - 0s 80ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1745/2000
6/6 [==============================] - 0s 80ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1746/2000
6/6 [==============================] - 0s 82ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1747/2000
6/6 [==============================] - 0s 80ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1748/2000
6/6 [==============================] - 0s 82ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1749/2000
6/6 [==============================] - 0s 80ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1750/2000
6/6 [==============================] - 0s 83ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1751/2000
6/6 [==============================] - 0s 82ms/step - loss: 4.3919e-09 - categorical_accuracy: 1.0000
Epoch 1752/2000
6/6 [==============================] - 0s 81ms/step - lo

6/6 [==============================] - 0s 79ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1814/2000
6/6 [==============================] - 0s 80ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1815/2000
6/6 [==============================] - 0s 82ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1816/2000
6/6 [==============================] - 0s 82ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1817/2000
6/6 [==============================] - 0s 82ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1818/2000
6/6 [==============================] - 0s 81ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1819/2000
6/6 [==============================] - 0s 82ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1820/2000
6/6 [==============================] - 0s 83ms/step - loss: 3.7645e-09 - categorical_accuracy: 1.0000
Epoch 1821/2000
6/6 [==============================] - 0s 80ms/step - lo

6/6 [==============================] - 0s 81ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1883/2000
6/6 [==============================] - 0s 82ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1884/2000
6/6 [==============================] - 0s 79ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1885/2000
6/6 [==============================] - 0s 81ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1886/2000
6/6 [==============================] - 0s 81ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1887/2000
6/6 [==============================] - 0s 80ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1888/2000
6/6 [==============================] - 0s 81ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1889/2000
6/6 [==============================] - 0s 81ms/step - loss: 3.1371e-09 - categorical_accuracy: 1.0000
Epoch 1890/2000
6/6 [==============================] - 0s 80ms/step - lo

6/6 [==============================] - 0s 81ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1952/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1953/2000
6/6 [==============================] - 0s 81ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1954/2000
6/6 [==============================] - 0s 81ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1955/2000
6/6 [==============================] - 0s 81ms/step - loss: 2.5097e-09 - categorical_accuracy: 1.0000
Epoch 1956/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1957/2000
6/6 [==============================] - 0s 79ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1958/2000
6/6 [==============================] - 0s 82ms/step - loss: 1.8823e-09 - categorical_accuracy: 1.0000
Epoch 1959/2000
6/6 [==============================] - 0s 81ms/step - lo

In [186]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_25 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_26 (LSTM)              (None, 64)                49408     
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 4)                 132       
                                                                 
Total params: 596708 (2.28 MB)
Trainable params: 59670

In [187]:
res = model.predict(X_test)
len(res)

1/1 [==============================] - 0s 263ms/step


10

In [190]:
actions[np.argmax(res[0])]

'hello'

In [191]:
actions[np.argmax(y_test[0])]

'hello'

In [192]:
model.save('test.keras')

In [193]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [194]:
yhat = model.predict(X_train)

6/6 [==============================] - 0s 24ms/step


In [195]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [196]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[141,   0],
        [  0,  49]],

       [[142,   0],
        [  0,  48]],

       [[143,   0],
        [  0,  47]],

       [[144,   0],
        [  0,  46]]], dtype=int64)

In [197]:
accuracy_score(ytrue, yhat)

1.0

In [198]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
#, (16,117,245),(21,23,245),(16,22,33),(16,117,22),(16,0,245),(23,117,245)
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [199]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

# Test the model

In [200]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from tensorflow.keras.models import load_model
model=load_model('test.keras')
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_25 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_26 (LSTM)              (None, 64)                49408     
                                                                 
 dense_24 (Dense)            (None, 64)                4160      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 4)                 132       
                                                                 
Total params: 596708 (2.28 MB)
Trainable params: 59670

In [203]:
sequence = []
sentence = []
threshold = 0.7


cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    prev_keypoints = None  # Store the previous keypoints
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
         
        draw_custom_landmarks(image, results)
        
        # Predict only when there's a significant change in keypoints
        keypoints = get_keypoints(results)
        
        if prev_keypoints is not None:
            # Calculate the Euclidean distance between the current and previous keypoints
            euclidean_distance = np.linalg.norm(np.array(prev_keypoints) - np.array(keypoints))
            #print(euclidean_distance)
            # You can set a threshold for the distance below which you don't make predictions
            if euclidean_distance > 0.1:
                sequence.append(keypoints)
                sequence = sequence[-30:]
        
        prev_keypoints = keypoints
        
        
        #predict
        
#         keypoints = get_keypoints(results)
#         sequence.append(keypoints)
#         sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sequence = []
        if res.any():
            if res[np.argmax(res)].any() > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                        
                else:
                    sentence.append(actions[np.argmax(res)])
                  

    
        if len(sentence) > 5:
            sentence = sentence[-5:]
        # Viz probabilities
        
        #image = prob_viz(res, actions, image, colors)
            
        
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Camera", image)
        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()
    

1/1 [==============================] - 0s 17ms/step


In [202]:
cap.release()
cv2.destroyAllWindows()